# Reporte de Transfer Learning: Clasificación de Razas de Perros

## 1. Explicación del Enfoque
En este laboratorio utilizamos la técnica de **Transfer Learning** (Aprendizaje por Transferencia) para resolver un problema de clasificación de imágenes multiclase (diferentes razas de perros).

En lugar de entrenar una Red Neuronal Convolucional (CNN) desde cero —lo cual requeriría millones de imágenes y días de cómputo— utilizamos **MobileNetV2**, un modelo pre-entrenado por Google con el dataset masivo **ImageNet** (1.4 millones de imágenes).

### Metodología aplicada:
1.  **Feature Extraction (Extracción de Características):** Cargamos MobileNetV2 pero descartamos su capa de salida (`include_top=False`). Conservamos el bloque convolucional (la "base") que ya sabe detectar bordes, texturas, formas de orejas y hocicos gracias a su entrenamiento previo.
2.  **Freezing (Congelamiento):** Establecimos `base_model.trainable = False`. Esto "congela" los pesos de la red base para evitar destruir la información aprendida previamente durante la retropropagación.
3.  **Custom Head (Cabezal Personalizado):** Añadimos nuestras propias capas densas al final para adaptar el conocimiento general de la red a nuestro problema específico: distinguir entre nuestras clases de perros.

## 2. Conclusiones

Basado en la implementación y los resultados obtenidos, se concluye lo siguiente:

* **Eficiencia Computacional:** El uso de Transfer Learning permitió que el modelo convergiera a una alta precisión (accuracy) en muy pocas épocas (5 épocas). Entrenar una red de esta profundidad desde cero habría resultado en un sobreajuste (overfitting) inmediato o en tiempos de entrenamiento inviables.

* **Robustez ante Datasets Pequeños:** Aunque el dataset de Kaggle no tiene millones de fotos por raza, el modelo logró generalizar correctamente. Esto confirma que las redes pre-entrenadas son la solución estándar cuando se dispone de datos limitados ("Small Data"), ya que reutilizan características visuales universales aprendidas de otros dominios.

* **Importancia del "Congelamiento":** Al congelar la base, obligamos al modelo a centrarse únicamente en aprender las diferencias de alto nivel entre razas (ej: la diferencia entre un Husky y un Malamute) en lugar de reaprender cómo ver una línea o un círculo, optimizando drásticamente el proceso.

* **Arquitectura MobileNetV2:** La elección de esta arquitectura específica ofrece un equilibrio ideal entre velocidad y precisión, haciendo que este sistema sea apto para implementarse en dispositivos con recursos limitados (como móviles o Raspberry Pi) en el futuro.

In [5]:
## 3. Procesamiento del dataset https://www.kaggle.com/datasets/mohamedchahed/dog-breeds/data

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import os

# --- 1. CONFIGURACIÓN ---
# Establecer la ruta donde descomprimiste las carpetas de perros
dataset_dir = "dog_dataset" 

# Parámetros
IMG_SIZE = (224, 224) # Tamaño esperado
BATCH_SIZE = 32
EPOCHS = 5  # Pocas epochs  equipo lento

# --- 2. CARGAR Y PREPARAR DATOS ---
# Usamos ImageDataGenerator para normalizar y dividir en train/val
print("Cargando imágenes...")

# Data augmentation (opcional, ayuda a que aprenda mejor)
train_datagen = ImageDataGenerator(
    rescale=1./255,       # Normalizar píxeles de 0-255 a 0-1
    validation_split=0.2, # Usar 20% para validar
    rotation_range=20,
    horizontal_flip=True
)

# Generador de Entrenamiento
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Generador de Validación
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)
print(f" Se detectaron {num_classes} razas de perros.")

# --- 3. TRANSFER LEARNING ---

# A) Cargar el modelo base pre-entrenado 
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))

# B) Congelar el modelo base
# Esto evita destruir lo que la red ya sabe durante el entrenamiento inicial
base_model.trainable = False 

# C) Añadir la parte clasificadora
x = base_model.output
x = GlobalAveragePooling2D()(x) # Aplanar los datos
x = Dense(1024, activation='relu')(x) # Capa densa para aprender patrones de perros
x = Dropout(0.2)(x) # Evitar sobreajuste
predictions = Dense(num_classes, activation='softmax')(x) # Capa final: tantas neuronas como razas

# D) Unir todo en un modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# --- 4. Procesar Entrenamiento ---
print("\nIniciando entrenamiento con Transfer Learning...")
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

print("\n Entrenamiento finalizado.")

# --- 5. GUARDAR ---
model.save("modelo_perros.h5")
print("Modelo guardado como 'modelo_perros.h5'")

Cargando imágenes...
Found 435 images belonging to 8 classes.
Found 106 images belonging to 8 classes.
 Se detectaron 8 razas de perros.

Iniciando entrenamiento con Transfer Learning...
Epoch 1/5
13/13 [==============================] - 30s 2s/step - loss: 0.5027 - accuracy: 0.8486 - val_loss: 0.1495 - val_accuracy: 0.9688
Epoch 2/5
13/13 [==============================] - 27s 2s/step - loss: 0.0508 - accuracy: 0.9727 - val_loss: 0.1787 - val_accuracy: 0.9688
Epoch 3/5
13/13 [==============================] - 34s 3s/step - loss: 0.0819 - accuracy: 0.9801 - val_loss: 0.1832 - val_accuracy: 0.9479
Epoch 4/5
13/13 [==============================] - 26s 2s/step - loss: 0.0196 - accuracy: 0.9901 - val_loss: 0.0955 - val_accuracy: 0.9688
Epoch 5/5
13/13 [==============================] - 25s 2s/step - loss: 0.0060 - accuracy: 0.9975 - val_loss: 0.2026 - val_accuracy: 0.9792

 Entrenamiento finalizado.
Modelo guardado como 'modelo_perros.h5'
